# Income Allocation AI model

In [3]:
import pandas as pd
import numpy as np

# Set seed for reproducibility
np.random.seed(42)

# Number of entries
n_entries = 10000

# Generate total income with different brackets (from 500 to 10000 cedis)
total_income = np.concatenate([
    np.random.randint(500, 2001, 2000),
    np.random.randint(2001, 4001, 2000),
    np.random.randint(4001, 6001, 2000),
    np.random.randint(6001, 8001, 2000),
    np.random.randint(8001, 10001, 2000)
])

# Generate percentages ensuring valid ranges
def generate_valid_percentages(n):
    essentials = np.random.uniform(30, 70, n)
    savings = np.random.uniform(5, 30, n)
    investments = np.random.uniform(5, 30, n)
    discretionary = 100 - (essentials + savings + investments)
    discretionary = np.clip(discretionary, 0, 100)  # Ensure no negative values
    return essentials, savings, investments, discretionary

essentials_percentage, savings_percentage, investments_percentage, discretionary_percentage = generate_valid_percentages(n_entries)

# Risk tolerance and financial status
risk_tolerance = np.random.choice(['Low', 'Medium', 'High'], n_entries)
financial_status = np.random.choice(['strain', 'stable', 'growth'], n_entries)

# Create DataFrame
data = pd.DataFrame({
    'total_income': total_income,
    'essentials_percentage': essentials_percentage.round(2),
    'savings_percentage': savings_percentage.round(2),
    'investments_percentage': investments_percentage.round(2),
    'discretionary_percentage': discretionary_percentage.round(2),
    'risk_tolerance': risk_tolerance,
    'financial_status': financial_status
})

# Save to CSV
data.to_csv('financial_data.csv', index=False)

print("Dataset generated and saved to 'financial_data.csv'.")


Dataset generated and saved to 'financial_data.csv'.


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

# Load data from a CSV file
data = pd.read_csv('financial_data.csv')  # Replace 'data.csv' with the path to your CSV file

# Calculate percentages
data['disposable_percentage'] = 100 - (data['essentials_percentage'] + data['discretionary_percentage'] + data['investments_percentage'])

# Define features and targets
X_status = data[['total_income', 'essentials_percentage', 'investments_percentage', 'discretionary_percentage', 'risk_tolerance']]
y_status = data['financial_status']

X_savings_investments = data[['total_income', 'disposable_percentage', 'financial_status', 'risk_tolerance']]
y_savings = data['savings_percentage']
y_investments = data['investments_percentage']




Predicted Financial Status: strain
Recommended Savings Percentage: 29.99%
Recommended Investments Percentage: 14.90%


In [ ]:
# Define preprocessing
preprocessor_status = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['total_income', 'essentials_percentage', 'investments_percentage', 'discretionary_percentage']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['risk_tolerance'])
    ]
)

preprocessor_savings_investments = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['total_income', 'disposable_percentage']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['financial_status', 'risk_tolerance'])
    ]
)

# Build pipelines for financial status and savings/investments
status_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_status),
    ('classifier', RandomForestClassifier(n_estimators=100))
])

savings_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_savings_investments),
    ('regressor', RandomForestRegressor(n_estimators=100))
])

investments_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_savings_investments),
    ('regressor', RandomForestRegressor(n_estimators=100))
])


In [ ]:
# Split the data
X_train_status, X_test_status, y_train_status, y_test_status = train_test_split(X_status, y_status, test_size=0.25, random_state=0)
X_train_savings, X_test_savings, y_train_savings, y_test_savings = train_test_split(X_savings_investments, y_savings, test_size=0.25, random_state=0)
X_train_investments, X_test_investments, y_train_investments, y_test_investments = train_test_split(X_savings_investments, y_investments, test_size=0.25, random_state=0)

# Train the models
status_pipeline.fit(X_train_status, y_train_status)
savings_pipeline.fit(X_train_savings, y_train_savings)
investments_pipeline.fit(X_train_investments, y_train_investments)

# Define user data
user_data = {
    'total_income': 10000,
    'amount_invested': 80,
    'amount_essentials': 10,
    'amount_discretionary': 10,
    'risk_tolerance': 'High'
}

# Calculate user percentages
user_data['essentials_percentage'] = user_data['amount_essentials'] / user_data['total_income'] * 100
user_data['investments_percentage'] = user_data['amount_invested'] / user_data['total_income'] * 100
user_data['discretionary_percentage'] = user_data['amount_discretionary'] / user_data['total_income'] * 100
user_data['disposable_percentage'] = 100 - (user_data['essentials_percentage'] + user_data['discretionary_percentage'] + user_data['investments_percentage'])


In [ ]:
# Convert user data to DataFrame for status prediction
user_df_status = pd.DataFrame([{
    'total_income': user_data['total_income'],
    'essentials_percentage': user_data['essentials_percentage'],
    'investments_percentage': user_data['investments_percentage'],
    'discretionary_percentage': user_data['discretionary_percentage'],
    'risk_tolerance': user_data['risk_tolerance']
}])

# Predict financial status
financial_status_prediction = status_pipeline.predict(user_df_status)[0]
user_data['financial_status'] = financial_status_prediction

# Convert user data to DataFrame for savings/investments prediction
user_df_savings_investments = pd.DataFrame([{
    'total_income': user_data['total_income'],
    'disposable_percentage': user_data['disposable_percentage'],
    'financial_status': user_data['financial_status'],
    'risk_tolerance': user_data['risk_tolerance']
}])

In [ ]:
# Predict savings and investments percentages
savings_predictions = savings_pipeline.predict(user_df_savings_investments)
investments_predictions = investments_pipeline.predict(user_df_savings_investments)

print(f"Predicted Financial Status: {financial_status_prediction}")
print(f"Recommended Savings Percentage: {savings_predictions[0]:.2f}%")
print(f"Recommended Investments Percentage: {investments_predictions[0]:.2f}%")

In [5]:
import joblib

# Save the pipelines
joblib.dump(status_pipeline, 'status_pipeline.pkl')
joblib.dump(savings_pipeline, 'savings_pipeline.pkl')
joblib.dump(investments_pipeline, 'investments_pipeline.pkl')


['investments_pipeline.pkl']